# import def func

In [1]:
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

from JohnsonUtil import LoggerFactory
log = LoggerFactory.log
# log_level = LoggerFactory.DEBUG
# log_level = LoggerFactory.INFO
log_level = LoggerFactory.ERROR
log.setLevel(log_level)
# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

log = LoggerFactory.log

In [2]:
def show_tdx_data(df, codelist=None, market_value='3', col=None,limit=20,orderby='percent'):
    col_src = [
        "name",'percent', "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    if orderby in df.columns:
        df=df.sort_values('percent', ascending=False)
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name][:limit]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name][:limit]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name][:limit]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

# from JSONData import tdx_hdf5_api as h5a
names = locals()
def get_tdx_all_now(resample='d',market_blk='all',showtable=True):
    top_now = tdd.getSinaAlldf(market=market_blk, vol=ct.json_countVol, vtype=ct.json_countType)
    if resample=='d' or resample=='3d':
        dl=ct.Resample_LABELS_Days[resample]
    elif  resample=='w':
        #200
        dl=ct.Resample_LABELS_Days[resample]
    else:
        #300
        dl=ct.Resample_LABELS_Days[resample]
    filter='y'
#     import pdb;pdb.set_trace()
    if not f'lastpTDX_DF_{resample}' in names:
        print(f'lastpTDX_DF_{resample} not in locals')
        top_all, lastpTDX_DF = tdd.get_append_lastp_to_df(top_now, dl=dl,resample=resample,showtable=showtable)
        names[f'lastpTDX_DF_{resample}'] = lastpTDX_DF
    else:
        print(f'lastpTDX_DF_{resample} in locals')
        print(eval(f'lastpTDX_DF_{resample}.shape'))
        top_all = tdd.get_append_lastp_to_df(top_now, eval(f'lastpTDX_DF_{resample}'))
    return top_all

def get_tdx_all_from_h5_old(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    # resample=resample
    # if resample=='d' or resample=='3d':
    #     dl='60'
    # elif  resample=='w':
    #     dl='200'
    # else:
    #     dl='300'
    if resample=='d' or resample=='3d':
        dl=ct.Resample_LABELS_Days[resample]
    elif  resample=='w':
        #200
        dl=ct.Resample_LABELS_Days[resample]
    else:
        #300
        dl=ct.Resample_LABELS_Days[resample]
        
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    if h5 is None:
        h5,tdx = tdd.get_append_lastp_to_df(sina_data.Sina().market('all'),dl=dl,resample=resample)
    return h5


# def get_tdx_all_from_h5_old(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
#     if end is not None:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
#             '_' + end.replace('-', '') + '_' + 'all'
#     else:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     #sina_monitor
#     h5_fname = 'tdx_last_df'
#     resample=resample
#     if resample=='d' or resample=='3d':
#         dl='60'
#     elif  resample=='w':
#         dl='200'
#     else:
#         dl='300'
#     filter='y'
#     h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
#     return h5

def write_code_to_blk(codew, blk='077',rewrite=False):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    if not rewrite:
        write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
        if write_blk == 'y' or write_blk == 'Y':
            hdf5_wri = cct.cct_raw_input(
                "Rewrite code [Y] or append [N](defalut:N):")
            if hdf5_wri == 'y' or hdf5_wri == 'Y':
                append_status = False
            else:
                append_status = True
            if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=append_status,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
            else:
                print("write error:%s block_path:%s" % (len(codew), block_path))
    else:
        if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=False,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write error:%s block_path:%s" % (len(codew), block_path))

def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    if not os.path.exists(fpath):
        print(f'fpath is not exists')
        return None
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if len(store.keys()) > 0 and root is None and store is not None:
        root = store.keys()[0].replace("/", "")
        df = store[root]
        df = apply_col_toint(df)
    else:
        df = None
    store.close()
    return df

In [3]:
f'lastpTDX_DF_d' in names
f'lastpTDX_DF_d' in locals()
# eval(f'lastpTDX_DF_d')

False

False

## init data

### get_tdx_all

In [4]:
# h5_day=get_tdx_all_now(resample='d')

### sina

### sina_all

In [5]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

('2025-09-17',
 code
 600186    莲花控股
 Name: name, dtype: object)

In [6]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-09/17/25 21:23:07'

code
600776   2025-09-17 15:00:04
002434   2025-09-17 15:00:00
Name: ticktime, dtype: datetime64[ns]

383.12369336684543

#### 实时成交

In [7]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
if h5 is not None:
    h5.shape
    h5code='603019'
    h5.loc[h5code][-3:]
    code_muti = '603019'
    # h5.loc[code_muti][:2]

    freq = 'D'
    # freq = '5M'
    startime = '09:25:00'
    endtime = '15:01:00'

    time_ratio = cct.get_work_time_ratio()
    time_ratio
    run_col = ['close', 'volume']
    # run_col = ['close']
    mdf = cct.get_limit_multiIndex_freq(
        h5, freq=freq.upper(),
        col=run_col, start=startime, end=endtime, code=None)
    allvol = mdf.volume.sum()
    allvol = round(allvol / 10 / 1000 / 1000,2)
    print("总量:%s 亿"%(allvol))
    mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
    mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
    #全局占比
    mdf['volume'] = mdf.amount.apply(
        lambda x: round(x / allvol * 100 / time_ratio, 2))
    mdf=mdf.reset_index().set_index('code')
    mdf.loc[h5code]
    str(h5.index.get_level_values(1)[-1])[:10]
else:
    mdf = None

sina_MultiD_path:G:\sina_MultiIndex_data.h5
['/all_30']
G:\sina_MultiIndex_data.h5


(634034, 6)

,close,high,low,llastp,volume,lastbuy
ticktime,,,,,,
2025-09-17 14:59:53,108.38,109.75,105.25,111.5,117269514,108.75
2025-09-17 15:00:02,108.50,109.75,105.25,111.5,118721114,108.75
2025-09-17 15:00:02,108.50,109.75,105.25,111.5,118721114,108.50


1.0

总量:14434.21 亿


ticktime    2025-09-18 00:00:00
close                108.686885
volume                     0.89
vol                      118.72
amount                   129.03
Name: 603019, dtype: object

'2025-09-17'

#### 读取实时成交个股

In [8]:
if mdf is not None:
    nowdf = mdf.sort_values(by=['volume'], ascending=[0])
    # nowdf.shape, df.shape, code_idx.shape
    # top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = get_sina_data_cname(top20_nd)
    nowdf.shape, top20_nd

((5188, 5),
         volume  name
 code                
 300476    1.48  胜宏科技
 300750    1.42  宁德时代
 300059    1.33  东方财富
 688981    1.24  中芯国际
 002050    1.06  三花智控
 601138    1.02  工业富联
 688256    0.98   寒武纪
 300274    0.91  阳光电源
 300308    0.91  中际旭创
 603019    0.89  中科曙光
 600580    0.89  卧龙电驱
 300502    0.83   新易盛
 688041    0.83  海光信息
 300803    0.79   指南针
 002594    0.75   比亚迪
 002475    0.71  立讯精密
 600410    0.67  华胜天成
 600030    0.61  中信证券
 300115    0.56  长盈精密
 601689    0.55  拓普集团)

##### 合并实时成交量及SinaDM

In [9]:
# mdf[:1]
if mdf is not None:
    sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
else:
    sinadm = sina_dm
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)


## resample 多周期

### filterKey!!!

In [10]:
filter_query={'2周期上冲上轨And涨幅新高': 'lasth1d > upper1 and lasth2d >upper2 and high >boll and percent > maxp',
             '3周期新高上轨':'lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3',
             '3周期新高大于Ma5': 'lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d',
             '新高上upper':'lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper',
            }   
filter_keys=list(filter_query.keys())

In [11]:
def get_filter(idx=0):
    filter_key=filter_query[filter_keys[idx]]
    print(f'{filter_keys[idx]}: {filter_key}')
    return filter_key

In [12]:
filterIDX=3
# h5_3d=None,h5_d=None,h5_w=None,h5_m=None

### resample 3d

In [13]:
# h5_3d = get_tdx_all_from_h5(resample='3d') if h5_3d is None else h5_3d
h5_3d = get_tdx_all_now(resample='3d')
# tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

initdx :0 b1>:5197 it:0.3 lastpTDX_DF_3d not in locals
fname: tdx_last_df keys:['/low_3d_120_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


[2025-09-17 21:23:11,069] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4587): tdx Out:1 code:['300280']


tdxdataT:0.8
DLTDXE:0.80 

In [14]:
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select = tdx_3d.query('close > lastp1d and percent < 3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and (lasth2d > upper2 or lasth3d > upper3) and lasth2d < hmax')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and lasth3d > lasth4d and lastl1d > lastl2d and lastl2d > lastl3d')
# t3d_code = h5_3d.query(f'{get_filter(filterIDX)}')
t3d_code = h5_3d.query('lasth1d > ma51d and lasth2d > ma52d and lasth3d > ma53d and ma51d > ma201d and ma52d >ma202d and ma53d > ma203d and macd > 0 and macddif > macddea')
show_tdx_data(t3d_code,limit=10)

Count:79 251


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
000981,山子高科,10.12,136,2025-04-28,3.70,2.77,2.00,96.0,3.70,1.68,69.0,24.0
600376,首开股份,10.04,723,2025-04-07,7.34,5.51,2.27,49.0,7.34,2.26,84.0,79.0
603786,科博达,9.98,1,2025-06-20,76.50,61.66,49.91,84.0,76.50,50.59,46.0,-22.0
002059,云南旅游,9.96,391,2025-05-23,6.07,5.37,5.01,43.0,6.07,5.02,26.0,41.0
603843,*ST正平,4.98,0,2025-05-07,4.22,3.65,2.57,102.0,4.22,2.17,40.0,40.0
871634,新威凌,4.17,0,2025-04-07,35.22,34.12,26.11,29.0,33.50,20.09,21.0,11.0
600026,中远海能,3.50,81,2025-04-07,12.73,12.09,9.99,28.0,12.70,9.68,89.0,-1.0
600007,中国国贸,3.43,0,2025-04-07,22.33,21.52,20.20,35.0,22.28,19.77,8.0,9.0
600103,青山纸业,2.95,1094,2025-04-07,4.54,4.18,2.12,39.0,4.51,2.00,99.0,11.0


In [15]:
if '301038' in t3d_code.index:
    h5_3d.loc['301038',['lastdu4','boll','ral','fib','percmax','macd','macddif','macddea','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

In [16]:
# t3d_code = tdx_3d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# t3d_code = h5_3d.query(f'{get_filter(1)}')
# show_tdx_data(t3d_code)

### resample W

In [17]:
# h5_w = get_tdx_all_from_h5(resample='w') if h5_w is None else h5_w

h5_w = get_tdx_all_now(resample='w') 
# tdx_w = cct.combine_dataFrame(h5_w,sinadm)

initdx :0 b1>:5197 it:0.3 lastpTDX_DF_w not in locals
fname: tdx_last_df keys:['/low_3d_120_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [18]:
# tw_code = tdx_w.query('lasth1d > lasth2d > lasth3d and close >= high4')
# tw_code = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
# tw_code = tdx_w.query ('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
# tw_code = tdx_w.query(f'{get_filter(0)}')
# tw_code = tdx_w.query(f'{get_filter(filterIDX)}')
tw_code = h5_w.query(f'{get_filter(filterIDX)}')
# tw_code = h5_w.query('lastp2d > ma51d and lastp3d > ma51d and macd > 0 and macddif > macddea')
show_tdx_data(tw_code,limit=10)

新高上upper: lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper
Count:449 251


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
300959,线上线下,18.94,1,2025-04-11,82.00,48.38,30.17,76.0,81.94,30.17,-22.0,5.0
301488,豪恩汽电,14.21,0,2025-01-03,218.00,159.12,46.50,39.0,217.12,49.97,46.0,26.0
300684,中石科技,14.15,24,2025-01-03,44.62,36.75,16.38,54.0,44.28,20.16,-37.0,41.0
300469,信息发展,10.85,1,2025-01-10,88.81,68.88,24.25,54.0,88.62,20.38,64.0,71.0
601388,怡球资源,10.14,1681,2025-04-11,3.26,2.86,2.06,43.0,3.26,2.06,28.0,5.0
000981,山子高科,10.12,136,2025-04-25,3.70,2.77,1.52,94.0,3.70,1.71,39.0,21.0
600162,香江控股,10.08,65,2025-06-20,2.62,1.96,1.47,94.0,2.62,1.56,1.0,5.0
600376,首开股份,10.04,723,2025-01-24,7.34,5.51,2.12,49.0,7.34,2.24,78.0,14.0
000737,北方铜业,10.03,1217,2024-12-27,15.25,13.63,7.37,23.0,14.79,7.79,15.0,-15.0


In [19]:
if '301038' in tw_code.index:
    h5_w.loc['301038',['macd','macddif','macddea','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

### resample M

In [20]:
# h5_m = get_tdx_all_from_h5(resample='m') if h5_m is None else h5_m
h5_m = get_tdx_all_now(resample='m')
# tdx_m = cct.combine_dataFrame(h5_m,sinadm)

initdx :0 b1>:5197 it:0.2 lastpTDX_DF_m not in locals
fname: tdx_last_df keys:['/low_3d_120_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


In [21]:
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastp1d > ma51d')
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
#月线站上,3月反弹,2月新高
tm_code = h5_m.query('lastl1d > lastl2d > lastl3d and lasth1d > lastp2d > lastp3d and  ma51d > ma52d and macd > 0 and macddif > macddea')   
# tm_code = h5_m.query(f'{get_filter(filterIDX)}')
show_tdx_data(tm_code,limit=10)

Count:390 251


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
688135,利扬芯片,19.94,17,2024-08-30,33.62,0.0,12.20,24.0,32.00,13.35,36.0,29.0
301488,豪恩汽电,14.21,0,2024-08-30,218.00,0.0,43.06,31.0,217.12,45.19,6.0,44.0
300843,胜蓝股份,12.11,21,2024-08-30,60.19,0.0,21.17,-4.0,60.47,22.61,-7.0,-18.0
301502,华阳智能,10.51,0,2025-04-30,57.50,0.0,28.95,-5.0,57.34,28.95,8.0,39.0
600089,特变电工,10.08,2303,2025-04-30,16.16,0.0,10.60,11.0,16.16,10.60,29.0,13.0
603219,富佳股份,10.06,21,2024-10-31,21.44,0.0,11.42,5.0,21.44,12.31,5.0,-16.0
001234,泰慕士,10.05,0,2024-08-30,33.19,0.0,13.48,39.0,33.19,14.13,28.0,15.0
600376,首开股份,10.04,723,2024-08-30,7.34,0.0,1.94,44.0,7.34,1.96,29.0,2.0
000737,北方铜业,10.03,1217,2024-12-31,15.25,0.0,7.37,19.0,14.79,7.69,1.0,-10.0


In [22]:
if '301038' in tm_code.index:
    h5_m.loc['301038',['lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]
    
# h5_m.loc['301038',['lasth1d','lastp2d','lastp3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]    

### resample D

In [23]:
# h5_d = get_tdx_all_from_h5(resample='d') if h5_d is None else h5_d
h5_d = get_tdx_all_now(resample='d')
# tdx_d = cct.combine_dataFrame(h5_d,sinadm)
h5_d.shape

initdx :0 b1>:5197 it:0.2 lastpTDX_DF_d not in locals
fname: tdx_last_df keys:['/low_3d_120_y_all', '/low_d_60_y_all', '/low_m_300_y_all', '/low_w_200_y_all']


(5178, 251)

In [24]:
# td_code = tdx_d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# td_code = tdx_d.query('ma51d > ma10d and ral > 3 and lastp1d >ma20d and low > lastl1d and vol > 2' )
# td_code = tdx_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')

# td_code = h5_d.query(f'{get_filter(filterIDX)}')
# td_code = h5_d.query(f'lasth1d > lasth2d and close > lastp1d and ral > 10 and fib > 0 and boll > 0')
td_code = h5_d.query(f'lasth1d > lasth2d  and ral > 10 and fib > 0 and boll > 0  and percent > lastdu4 \
and close >high4  and (low > open*0.99 or (per1d > 4 and low >= lastp1d))')

show_tdx_data(td_code,limit=20)
# if len(td_code) < 30:
#     write_code_to_blk(td_code.sort_values('percent', ascending=False).index,'065',rewrite=True)

Count:8 251


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
603013,亚普股份,9.98,18,2025-06-26,27.55,25.95,20.62,9.0,27.55,16.45,-28.0,14.0
836270,天铭科技,9.03,0,2025-07-09,22.45,23.98,16.69,-24.0,22.53,16.70,-10.0,-4.0
603486,科沃斯,7.01,0,2025-06-27,105.00,100.44,84.50,8.0,105.12,54.44,9.0,19.0
605222,起帆电缆,6.28,3,2025-06-26,19.11,17.77,15.64,29.0,19.03,14.83,-19.0,25.0
688683,莱尔科技,5.98,0,2025-07-11,38.28,36.50,26.41,22.0,38.28,24.06,13.0,21.0
603699,纽威股份,5.95,1,2025-07-31,37.38,36.34,32.00,5.0,37.25,28.77,18.0,24.0
002531,天顺风能,5.68,145,2025-07-31,7.81,7.72,6.51,1.0,7.76,6.48,-14.0,-3.0
600029,南方航空,2.98,524,2025-07-15,6.23,6.14,5.68,25.0,6.23,5.65,-6.0,-19.0


In [25]:
if '301038' in td_code.index:
    h5_d.loc['301038',['lastdu4','close','boll','ral','fib','percmax','macd','macddif','macddea','macdlast1','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

#### filter test

In [26]:
# h5_t_code=h5_w.query('lasth1d > upper1 and lasth1d >lasth2d   and high >boll and percent > maxp')
# h5_t_code=h5_d.query('lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d and lastp2d < upper2 and lasth1d >upper')
code_3f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index))

show_tdx_data(t3d_code.loc[code_3f],limit=10)

Count:1 251


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
600376,首开股份,10.04,723,2025-04-07,7.34,5.51,2.27,49.0,7.34,2.26,84.0,79.0


In [27]:
# td_code2 = tdx_d.query(f'{get_filter(2)}')
# td_code2 = h5_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')
# show_tdx_data(td_code2,limit=10)


#### 多周期筛选

In [28]:
for co in ['tm_code','tw_code','t3d_code','td_code']:
    print(f'{co}:{len(eval(co))}')
# code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  & set(td_code.index))
code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  )
# code_f=  list(set(tm_code.index) & set(tw_code.index) & set(td_code.index))
# code_f =td_code.loc[code_f].sort_values('percent', ascending=False).index.tolist()
code_f =t3d_code.loc[code_f].sort_values('percent', ascending=False).index.tolist()

if len(code_f) > 0:
    show_tdx_data(t3d_code.loc[code_f],limit=10)
else:
    print('no code')

tm_code:390
tw_code:449
t3d_code:79
td_code:8
Count:1 251


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
600376,首开股份,10.04,723,2025-04-07,7.34,5.51,2.27,49.0,7.34,2.26,84.0,79.0


#### check data

In [29]:
'300998' in code_f
code_check = [co for co in code_f if co in h5_3d.index]
len(code_check)

False

1

In [30]:
show_tdx_data(t3d_code.loc[code_f])

Count:1 251


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
600376,首开股份,10.04,723,2025-04-07,7.34,5.51,2.27,49.0,7.34,2.26,84.0,79.0


### Write toblk

In [31]:
# write_code_to_blk(code_f,'077',rewrite=False)